In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from ollama import Client
client = Client(host="http://localhost:11434")
model_name = "qwen2.5:7b"


In [14]:
from src.agentic.tool import ToolAgent
from functions import summation, multiplication

In [15]:
agent = ToolAgent(client=client, toolkit=[summation, multiplication], model="qwen2.5:7b")

In [16]:
response = agent.start(message="Can you multiple 25 by 12 and sum 3 with 6 and search what is an LLM on google.com?")

In [17]:
response

"Sure, the result of multiplying 25 by 12 is 300, and the sum of 3 and 6 is 9. \n\nIf you're interested in knowing what an LLM (Large Language Model) is, I can provide a brief explanation or perform a web search for more detailed information. Would you like to proceed with that?"

In [22]:
from src.tool import Function, Tool, ToolKit
from typing import List

In [23]:
def sum2(a: float, b: float) -> float:
    return a + b

def sub(a: float, b: float) -> float:
    return a - b

def list_mul(a: int, b: List = [1,2,3]) -> List:
    return a * b

In [24]:
sum2_func = Function(sum2)
sub_func = Function(sub)

In [25]:
sub_func

Function(name=sub, signature=(a: float, b: float) -> float, doc=None)

In [26]:
sum2_func.run({'a': 2, 'b': 3})

5.0

In [27]:
sub_func.run({'a': 2, 'b': 3})

-1.0

In [28]:
sum2_tool = Tool(func=sum2)

In [29]:
sum2_tool.schema

{'name': 'sum2',
 'doc': None,
 'parameters': {'properties': {'a': {'type': 'float'},
   'b': {'type': 'float'}}}}

In [30]:
sum2_tool.run({'a': 2, 'b': 3})

5.0

In [31]:
assert sum2_tool.run({'a': 2, 'b': 3}) == sum2_func.run({'a': 2, 'b': 3})

In [32]:
sum2_tool.run({'a': 2, 'b': 3})

5.0

In [33]:
tk = ToolKit(tools=[sum2, sub, list_mul])

In [34]:
tk.tools

[Tool(name=sum2, signature=(a: float, b: float) -> float, doc=None),
 Tool(name=sub, signature=(a: float, b: float) -> float, doc=None),
 Tool(name=list_mul, signature=(a: int, b: List = [1, 2, 3]) -> List, doc=None)]

In [35]:
assert tk.tools[0].run({'a': 2, 'b': 3}) == sum2_tool.run({'a': 2, 'b': 3}) == sum2_func.run({'a': 2, 'b': 3})

In [36]:
assert tk.get_tool_by_name('sum2').run({'a': 2, 'b': 3}) == sum2_tool.run({'a': 2, 'b': 3}) == sum2_func.run({'a': 2, 'b': 3})

In [40]:
from src.tool import ToolCallProcessor
# Example usage
processor = ToolCallProcessor("""
<tool_call>
{"name": "summation", "arguments": {"x": 3, "y": 6}, "id": 2}
</tool_call>
<tool_call>
{"name": "search", "arguments": {"query": "what is an LLM on google.com"}, "id": 3}
</tool_call>
""")


In [41]:
for call in processor.calls:
    print(call)

name='summation' arguments={'x': 3, 'y': 6} id=2
name='search' arguments={'query': 'what is an LLM on google.com'} id=3
